In [1]:
from dotenv import load_dotenv
from tqdm import tqdm
from io import BytesIO
import tarfile
load_dotenv()
import os

from pyro_risks.utils.s3 import S3Bucket

s3 = S3Bucket(
	bucket_name="risk", 
	endpoint_url=os.environ["S3_ENDPOINT_URL"], 
	region_name=os.environ["S3_REGION"],
	aws_access_key_id=os.environ["S3_ACCESS_KEY"], 
	aws_secret_key=os.environ["S3_SECRET_KEY"]
)

zip_obj = s3.bucket.Object(key='datacube/datasets.tar.gz')

In [2]:
compressed_file = BytesIO(zip_obj.get()['Body'].read())

In [3]:
uncompressed_file = tarfile.open(None, 'r:gz', fileobj=compressed_file)

In [ ]:
for tarinfo in tqdm(uncompressed_file):
    if tarinfo.isreg():
        s3.s3_client.upload_fileobj(
            uncompressed_file.extractfile(tarinfo), 
            Bucket='risk', 
            Key=f"datacube/raw{tarinfo.name}"
        )